# Keras 预训练模型使用指南

预训练模型是在大规模数据集（如ImageNet）上训练好的深度学习模型。
使用预训练模型的优势：

1. **节省训练时间** - 无需从头训练数百万参数
2. **更好的泛化** - 已学习到通用的视觉特征
3. **小数据集也能用** - 通过迁移学习在小数据上获得好效果

本教程涵盖：
- Keras Applications模块介绍
- 加载和使用预训练ResNet50
- 图像预处理与推理
- 预测结果解码

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

print(f"TensorFlow版本: {tf.__version__}")
print(f"Keras版本: {keras.__version__}")

## 第一部分：Keras Applications 模块

### 1.1 可用的预训练模型

`tf.keras.applications` 提供多种预训练模型：

| 模型 | 参数量 | Top-1准确率 | 特点 |
|-----|--------|------------|------|
| VGG16/19 | 138M/144M | 71.3%/71.3% | 经典架构，参数多 |
| ResNet50/101/152 | 25M/44M/60M | 74.9%/76.4%/76.6% | 残差连接 |
| InceptionV3 | 24M | 77.9% | 多尺度特征 |
| Xception | 23M | 79.0% | 深度可分离卷积 |
| MobileNetV2 | 3.4M | 71.3% | 轻量级，移动端 |
| EfficientNetB0-B7 | 5M-66M | 77.1%-84.3% | 最优效率 |

In [ ]:
# 加载预训练的ResNet50模型
# weights='imagenet' 表示加载在ImageNet上预训练的权重
# include_top=True 表示包含最后的分类层（1000类）

print("正在加载ResNet50预训练模型...")
model = keras.applications.ResNet50(
    weights='imagenet',
    include_top=True,
    input_shape=(224, 224, 3)
)

print(f"模型加载完成")
print(f"总参数量: {model.count_params():,}")
print(f"输入形状: {model.input_shape}")
print(f"输出形状: {model.output_shape}")

## 第二部分：准备输入图像

### 2.1 预处理要求

不同模型有不同的预处理要求。ResNet50的输入需要：
1. 图像尺寸: 224×224
2. 像素值范围: 使用 `preprocess_input` 函数处理
3. 通道顺序: RGB

In [ ]:
# 加载示例图像
from sklearn.datasets import load_sample_image

# 加载示例图像（scikit-learn自带）
china = load_sample_image("china.jpg")
flower = load_sample_image("flower.jpg")

# 创建批次
images = np.array([china, flower], dtype=np.float32)

print(f"原始图像形状: {images.shape}")
print(f"像素值范围: [{images.min():.0f}, {images.max():.0f}]")

In [ ]:
# 调整图像尺寸
# ResNet50要求输入尺寸为224×224
images_resized = tf.image.resize(images, [224, 224])

print(f"调整后图像形状: {images_resized.shape}")

# 应用模型特定的预处理
# preprocess_input会进行通道标准化（减均值等）
inputs = keras.applications.resnet50.preprocess_input(images_resized)

print(f"预处理后像素值范围: [{inputs.numpy().min():.2f}, {inputs.numpy().max():.2f}]")

In [ ]:
# 可视化原始图像
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].imshow(china)
axes[0].set_title(f'China - 原始尺寸: {china.shape[:2]}')
axes[0].axis('off')

axes[1].imshow(flower)
axes[1].set_title(f'Flower - 原始尺寸: {flower.shape[:2]}')
axes[1].axis('off')

plt.tight_layout()
plt.show()

## 第三部分：进行预测

### 3.1 模型推理

In [ ]:
# 进行预测
print("正在进行预测...")
predictions = model.predict(inputs, verbose=0)

print(f"预测输出形状: {predictions.shape}")
print(f"每张图像有 {predictions.shape[1]} 个类别概率")

In [ ]:
# 解码预测结果
# decode_predictions返回Top-K预测结果，包含(类别ID, 类别名, 概率)

decoded_predictions = keras.applications.resnet50.decode_predictions(
    predictions, top=5
)

image_names = ['China (风景)', 'Flower (花朵)']

for i, (image_name, preds) in enumerate(zip(image_names, decoded_predictions)):
    print(f"\n{image_name} 的Top-5预测:")
    print("-" * 50)
    for rank, (class_id, class_name, prob) in enumerate(preds, 1):
        print(f"  {rank}. {class_name:20s} - {prob*100:5.2f}%")

## 第四部分：使用其他预训练模型

### 4.1 MobileNetV2 - 轻量级模型

In [ ]:
# 加载MobileNetV2
print("正在加载MobileNetV2...")
mobilenet = keras.applications.MobileNetV2(
    weights='imagenet',
    include_top=True
)

print(f"MobileNetV2 参数量: {mobilenet.count_params():,}")

# MobileNetV2的预处理
mobile_inputs = keras.applications.mobilenet_v2.preprocess_input(
    images_resized.numpy().copy()
)

# 预测
mobile_preds = mobilenet.predict(mobile_inputs, verbose=0)
mobile_decoded = keras.applications.mobilenet_v2.decode_predictions(mobile_preds, top=3)

print("\nMobileNetV2 预测结果:")
for i, (image_name, preds) in enumerate(zip(image_names, mobile_decoded)):
    print(f"\n{image_name}:")
    for class_id, class_name, prob in preds:
        print(f"  {class_name}: {prob*100:.2f}%")

### 4.2 提取特征而非分类

设置 `include_top=False` 可以获取特征提取器，用于迁移学习

In [ ]:
# 加载不含分类头的模型作为特征提取器
feature_extractor = keras.applications.ResNet50(
    weights='imagenet',
    include_top=False,          # 不包含分类层
    input_shape=(224, 224, 3),
    pooling='avg'               # 添加全局平均池化
)

print(f"特征提取器输出形状: {feature_extractor.output_shape}")

# 提取特征
features = feature_extractor.predict(inputs, verbose=0)
print(f"提取的特征形状: {features.shape}")
print(f"每张图像得到 {features.shape[1]} 维特征向量")

In [ ]:
# 计算两张图像特征的相似度
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(features[0:1], features[1:2])[0, 0]
print(f"两张图像的特征相似度（余弦）: {similarity:.4f}")

## 第五部分：模型对比

In [ ]:
# 对比不同模型的参数量和推理时间
import time

models_info = {
    'ResNet50': keras.applications.ResNet50,
    'MobileNetV2': keras.applications.MobileNetV2,
    'VGG16': keras.applications.VGG16,
}

print("模型对比:")
print("-" * 60)
print(f"{'模型名称':<15} {'参数量':>12} {'推理时间(ms)':>15}")
print("-" * 60)

for name, model_fn in models_info.items():
    # 加载模型
    temp_model = model_fn(weights='imagenet', include_top=True)
    
    # 准备输入（每个模型有自己的预处理）
    if 'mobilenet' in name.lower():
        temp_input = keras.applications.mobilenet_v2.preprocess_input(
            images_resized.numpy().copy()
        )
    elif 'vgg' in name.lower():
        temp_input = keras.applications.vgg16.preprocess_input(
            images_resized.numpy().copy()
        )
    else:
        temp_input = keras.applications.resnet50.preprocess_input(
            images_resized.numpy().copy()
        )
    
    # 测量推理时间（预热后）
    temp_model.predict(temp_input[:1], verbose=0)  # 预热
    
    start = time.time()
    for _ in range(5):
        temp_model.predict(temp_input[:1], verbose=0)
    elapsed = (time.time() - start) / 5 * 1000
    
    print(f"{name:<15} {temp_model.count_params():>12,} {elapsed:>15.2f}")
    
    # 清理内存
    del temp_model
    keras.backend.clear_session()

## 总结

### 使用预训练模型的关键步骤

1. **选择合适的模型** - 根据精度需求和计算资源选择
2. **正确预处理** - 使用对应模型的 `preprocess_input` 函数
3. **调整输入尺寸** - 大多数模型需要 224×224
4. **解码预测** - 使用 `decode_predictions` 获取可读结果

### 选择建议

| 场景 | 推荐模型 |
|-----|----------|
| 最高精度 | EfficientNetB7, ResNet152 |
| 平衡精度与速度 | ResNet50, EfficientNetB0 |
| 移动端/边缘设备 | MobileNetV2, MobileNetV3 |
| 迁移学习基础 | ResNet50, VGG16 |

### 下一步

- 迁移学习：在预训练模型上微调
- 特征提取：用预训练模型提取图像特征
- 模型量化：压缩模型用于部署